In [9]:

import pickle
import os
os.environ["TF_CPP_MIN_VLOG_LEVEL"] = "2"
os.system("nvcc --version")
import os
from glob import glob
import tensorflow as tf
import tensorflow_io as tfio

from PIL import Image
import numpy as np


In [10]:
classNames = ['islet', 'epithelium', 'vasculature', 'fat', 'acini', 'stroma', 'nontissue', 'PanIN', 'nerves', 'immune', 'PDAC', 'black']
    
print(classNames)

['islet', 'epithelium', 'vasculature', 'fat', 'acini', 'stroma', 'nontissue', 'PanIN', 'nerves', 'immune', 'PDAC', 'black']


In [12]:
#_______________Create Tensorflow dataset______________

# Paths to training and validation datasets
pthDataset = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\02_23_2024'

# Define paths to training and validation directories
pthTrain = os.path.join(pthDataset, 'training')
pthValidation = os.path.join(pthDataset, 'validation')

# Get paths to training images and labels
train_images = sorted(glob(os.path.join(pthTrain, 'im', "*.tif")))
train_masks = sorted(glob(os.path.join(pthTrain, 'label', 'corrected_labels', "*.tif")))

# Get paths to validation images and labels
val_images = sorted(glob(os.path.join(pthValidation, 'im', "*.tif")))
val_masks = sorted(glob(os.path.join(pthValidation, 'label', 'corrected_labels', "*.tif")))

# Define constants
IMAGE_SIZE = 1024
BATCH_SIZE = 4
NUM_TRAIN_IMAGES = len(train_images)  # Number of training images
NUM_VAL_IMAGES = len(val_images)  # Number of validation images

NUM_CLASSES = len(classNames)  # Number of classes


def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tfio.experimental.image.decode_tiff(image)
        image = tf.image.rgb_to_grayscale(image)  # Convert to grayscale if it's not already
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tfio.experimental.image.decode_tiff(image)
        if tf.shape(image)[-1] == 1:  # If grayscale, convert to RGB
            image = tf.image.grayscale_to_rgb(image)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    return tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]

def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask

def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

NotImplementedError: in user code:

    File "C:\Users\Valentina\AppData\Local\Temp\ipykernel_3980\3584796732.py", line 43, in load_data  *
        image = read_image(image_list)
    File "C:\Users\Valentina\AppData\Local\Temp\ipykernel_3980\226861467.py", line 35, in read_image  *
        image = tfio.experimental.image.decode_tiff(image)
    File "C:\Users\Valentina\anaconda3\envs\CODA_tinaFlow\lib\site-packages\tensorflow_io\python\experimental\image_ops.py", line 87, in decode_tiff  **
        return core_ops.io_decode_tiff(contents, index, name=name)
    File "C:\Users\Valentina\anaconda3\envs\CODA_tinaFlow\lib\site-packages\tensorflow_io\python\ops\__init__.py", line 88, in __getattr__
        return getattr(self._load(), attrb)
    File "C:\Users\Valentina\anaconda3\envs\CODA_tinaFlow\lib\site-packages\tensorflow_io\python\ops\__init__.py", line 84, in _load
        self._mod = _load_library(self._library)
    File "C:\Users\Valentina\anaconda3\envs\CODA_tinaFlow\lib\site-packag

In [11]:
#_______________Create Tensorflow dataset______________

# Paths to training and validation datasets
pthDataset = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\02_23_2024'

# Define paths to training and validation directories
pthTrain = os.path.join(pthDataset, 'training')
pthValidation = os.path.join(pthDataset, 'validation')

# Get paths to training images and labels
train_images = sorted(glob(os.path.join(pthTrain, 'im', "*.tif")))
train_masks = sorted(glob(os.path.join(pthTrain, 'label', 'corrected_labels', "*.tif")))

# Get paths to validation images and labels
val_images = sorted(glob(os.path.join(pthValidation, 'im', "*.tif")))
val_masks = sorted(glob(os.path.join(pthValidation, 'label', 'corrected_labels', "*.tif")))

# Define constants
IMAGE_SIZE = 1024
BATCH_SIZE = 4
NUM_TRAIN_IMAGES = len(train_images)  # Number of training images
NUM_VAL_IMAGES = len(val_images)  # Number of validation images

NUM_CLASSES = len(classNames)  # Number of classes


def read_image(image_path, mask=False):
    # Read the image file
    with tf.io.gfile.GFile(image_path, 'rb') as file:
        img = Image.open(file)
        
        # Convert to numpy array
        image = np.array(img)
        
        # Handle different image types
        if mask:
            if len(image.shape) == 3 and image.shape[2] > 1:
                image = image[:,:,0]  # Take first channel if multi-channel
            image = image[:,:,np.newaxis]  # Add channel dimension if not present
        else:
            if len(image.shape) == 2:  # If grayscale, convert to RGB
                image = np.stack((image,)*3, axis=-1)
            elif image.shape[2] == 4:  # If RGBA, convert to RGB
                image = image[:,:,:3]
        
        # Resize the image
        image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
        
        # Normalize to [0, 1]
        image = tf.cast(image, tf.float32) / 255.0
        
        return image

# Modify the load_data function to use the new read_image function
def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask

def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)

print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

TypeError: in user code:

    File "C:\Users\Valentina\AppData\Local\Temp\ipykernel_3980\1738606374.py", line 56, in load_data  *
        image = read_image(image_list)
    File "C:\Users\Valentina\AppData\Local\Temp\ipykernel_3980\1738606374.py", line 30, in read_image  *
        img = Image.open(file)
    File "C:\Users\Valentina\anaconda3\envs\CODA_tinaFlow\lib\site-packages\PIL\Image.py", line 3437, in open  **
        fp.seek(0)

    TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
        1. tensorflow.python.lib.io._pywrap_file_io.BufferedInputStream(filename: str, buffer_size: int, token: tensorflow.python.lib.io._pywrap_file_io.TransactionToken = None)
    
    Invoked with: <tf.Tensor 'args_0:0' shape=() dtype=string>, 524288
